In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sample/Contributing.md
/kaggle/input/sample/errata.md
/kaggle/input/sample/README.md
/kaggle/input/sample/LICENSE.txt
/kaggle/input/sample/9781484241301.jpg
/kaggle/input/sample/.gitattributes
/kaggle/input/sample/chapter_9_NLP/NLP_PySpark.ipynb
/kaggle/input/sample/chapter_9_NLP/Sequence_Embeddings_PySpark.ipynb
/kaggle/input/sample/chapter_9_NLP/Movie_reviews.csv
/kaggle/input/sample/chapter_6_Random_Forests/affairs.csv
/kaggle/input/sample/chapter_6_Random_Forests/Random_Forests.ipynb
/kaggle/input/sample/chapter_5_Logistic_Regression/Logistic_Regression_Pyspark.ipynb
/kaggle/input/sample/chapter_5_Logistic_Regression/Log_Reg_dataset.csv
/kaggle/input/sample/chapter_2_Data_Processing/Data_processing_using_PySpark.ipynb
/kaggle/input/sample/chapter_2_Data_Processing/sample_data.csv
/kaggle/input/sample/chapter_4_Linear_Regression/Linear_regression_dataset.csv
/kaggle/input/sample/chapter_4_Linear_Regression/Linear_Regression.ipynb
/kaggle/input/sample/chapter_8_Recommen

In [2]:
!pip install pyspark 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317146 sha256=3e2e5690c3dce1a1e3a12d3923e1a70f85639a9a80f2b7f1961edf9d38694cf6
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [3]:
# import spark session 
from pyspark.sql import SparkSession

In [4]:
# Create Spark Object
spark = SparkSession.builder.appName("Data_PreProcessing").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/11 20:11:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.read.csv("/kaggle/input/sample/chapter_2_Data_Processing/sample_data.csv", header=True, inferSchema=True)

In [6]:
# Columns of Dataframe
df.columns

['ratings', 'age', 'experience', 'family', 'mobile']

In [7]:
# Check number of Columns
len(df.columns)

5

In [8]:
#Number of records in dataframe
df.count()

33

In [9]:
# Shape of dataset 
print(df.count(),len(df.columns))

33 5


In [10]:
# printSchema
df.printSchema()

root
 |-- ratings: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: double (nullable = true)
 |-- family: integer (nullable = true)
 |-- mobile: string (nullable = true)



In [11]:
# First few rows of dataframe
df.show(5)

+-------+---+----------+------+-------+
|ratings|age|experience|family| mobile|
+-------+---+----------+------+-------+
|      3| 32|       9.0|     3|   Vivo|
|      3| 27|      13.0|     3|  Apple|
|      4| 22|       2.5|     0|Samsung|
|      4| 37|      16.5|     4|  Apple|
|      5| 27|       9.0|     1|     MI|
+-------+---+----------+------+-------+
only showing top 5 rows



In [12]:
# Select 2 Columns
df.select('age','experience','mobile').show(5)

+---+----------+-------+
|age|experience| mobile|
+---+----------+-------+
| 32|       9.0|   Vivo|
| 27|      13.0|  Apple|
| 22|       2.5|Samsung|
| 37|      16.5|  Apple|
| 27|       9.0|     MI|
+---+----------+-------+
only showing top 5 rows



In [13]:
# Info about dataframe
df.describe().show()

23/05/11 20:11:57 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------------------+------------------+------------------+------+
|summary|           ratings|               age|        experience|            family|mobile|
+-------+------------------+------------------+------------------+------------------+------+
|  count|                33|                33|                33|                33|    33|
|   mean|3.5757575757575757|30.484848484848484|10.303030303030303|1.8181818181818181|  null|
| stddev|1.1188806636071336|  6.18527087180309| 6.770731351213326|1.8448330794164254|  null|
|    min|                 1|                22|               2.5|                 0| Apple|
|    max|                 5|                42|              23.0|                 5|  Vivo|
+-------+------------------+------------------+------------------+------------------+------+



In [14]:
from pyspark.sql.types import StringType, DoubleType, IntegerType
df.withColumn("age_after_10Years",(df.age+10)).show(10,False)

+-------+---+----------+------+-------+-----------------+
|ratings|age|experience|family|mobile |age_after_10Years|
+-------+---+----------+------+-------+-----------------+
|3      |32 |9.0       |3     |Vivo   |42               |
|3      |27 |13.0      |3     |Apple  |37               |
|4      |22 |2.5       |0     |Samsung|32               |
|4      |37 |16.5      |4     |Apple  |47               |
|5      |27 |9.0       |1     |MI     |37               |
|4      |27 |9.0       |0     |Oppo   |37               |
|5      |37 |23.0      |5     |Vivo   |47               |
|5      |37 |23.0      |5     |Samsung|47               |
|3      |22 |2.5       |0     |Apple  |32               |
|3      |27 |6.0       |0     |MI     |37               |
+-------+---+----------+------+-------+-----------------+
only showing top 10 rows



In [15]:
# cast as double (age)
df.withColumn("age_doubleFormat",(df.age.cast('double'))).show(10,False)

+-------+---+----------+------+-------+----------------+
|ratings|age|experience|family|mobile |age_doubleFormat|
+-------+---+----------+------+-------+----------------+
|3      |32 |9.0       |3     |Vivo   |32.0            |
|3      |27 |13.0      |3     |Apple  |27.0            |
|4      |22 |2.5       |0     |Samsung|22.0            |
|4      |37 |16.5      |4     |Apple  |37.0            |
|5      |27 |9.0       |1     |MI     |27.0            |
|4      |27 |9.0       |0     |Oppo   |27.0            |
|5      |37 |23.0      |5     |Vivo   |37.0            |
|5      |37 |23.0      |5     |Samsung|37.0            |
|3      |22 |2.5       |0     |Apple  |22.0            |
|3      |27 |6.0       |0     |MI     |27.0            |
+-------+---+----------+------+-------+----------------+
only showing top 10 rows



In [16]:
#filter the records 
df.filter(df['mobile']=='Vivo').show()

+-------+---+----------+------+------+
|ratings|age|experience|family|mobile|
+-------+---+----------+------+------+
|      3| 32|       9.0|     3|  Vivo|
|      5| 37|      23.0|     5|  Vivo|
|      4| 37|       6.0|     0|  Vivo|
|      5| 37|      13.0|     1|  Vivo|
|      4| 37|       6.0|     0|  Vivo|
+-------+---+----------+------+------+



In [17]:
df.filter(df["mobile"]=="Apple").select("ratings","age","experience").show(5,False)

+-------+---+----------+
|ratings|age|experience|
+-------+---+----------+
|3      |27 |13.0      |
|4      |37 |16.5      |
|3      |22 |2.5       |
|3      |37 |16.5      |
|4      |27 |6.0       |
+-------+---+----------+
only showing top 5 rows



In [18]:
df.filter(df["age"] == 32).show()

+-------+---+----------+------+------+
|ratings|age|experience|family|mobile|
+-------+---+----------+------+------+
|      3| 32|       9.0|     3|  Vivo|
|      2| 32|      16.5|     2|  Oppo|
+-------+---+----------+------+------+



In [19]:
# Filter with multiple conditions
df.filter(df['age']==27).filter(df['mobile']=='Oppo').show()

+-------+---+----------+------+------+
|ratings|age|experience|family|mobile|
+-------+---+----------+------+------+
|      4| 27|       9.0|     0|  Oppo|
|      2| 27|       6.0|     2|  Oppo|
|      2| 27|       6.0|     2|  Oppo|
+-------+---+----------+------+------+



In [20]:
df.filter(df['mobile']=='Apple').filter(df['experience']>10.0).show()

+-------+---+----------+------+------+
|ratings|age|experience|family|mobile|
+-------+---+----------+------+------+
|      3| 27|      13.0|     3| Apple|
|      4| 37|      16.5|     4| Apple|
|      3| 37|      16.5|     5| Apple|
|      3| 37|      16.5|     5| Apple|
+-------+---+----------+------+------+



In [21]:
# Select Distinct mobile 
df.select("mobile").distinct().show()

+-------+
| mobile|
+-------+
|     MI|
|   Oppo|
|Samsung|
|   Vivo|
|  Apple|
+-------+



In [22]:
# Count distinct mobile
df.select("mobile").distinct().count()

5

# **GroupBy**

In [23]:
df.groupBy("age").count().orderBy('count',ascending=False).show(5,False)

+---+-----+
|age|count|
+---+-----+
|27 |14   |
|37 |10   |
|22 |5    |
|32 |2    |
|42 |2    |
+---+-----+



In [24]:
df.groupBy("mobile").count().orderBy("count",ascending=False).show()

+-------+-----+
| mobile|count|
+-------+-----+
|     MI|    8|
|   Oppo|    7|
|  Apple|    7|
|Samsung|    6|
|   Vivo|    5|
+-------+-----+



In [25]:
df.groupBy("mobile").mean().show()

+-------+------------------+------------------+------------------+------------------+
| mobile|      avg(ratings)|          avg(age)|   avg(experience)|       avg(family)|
+-------+------------------+------------------+------------------+------------------+
|     MI|               3.5|            30.125|           10.1875|             1.375|
|   Oppo| 2.857142857142857|28.428571428571427|10.357142857142858|1.4285714285714286|
|Samsung| 4.166666666666667|28.666666666666668| 8.666666666666666|1.8333333333333333|
|   Vivo|               4.2|              36.0|              11.4|               1.8|
|  Apple|3.4285714285714284|30.571428571428573|              11.0|2.7142857142857144|
+-------+------------------+------------------+------------------+------------------+



In [26]:
df.groupBy("mobile").sum().show()

+-------+------------+--------+---------------+-----------+
| mobile|sum(ratings)|sum(age)|sum(experience)|sum(family)|
+-------+------------+--------+---------------+-----------+
|     MI|          28|     241|           81.5|         11|
|   Oppo|          20|     199|           72.5|         10|
|Samsung|          25|     172|           52.0|         11|
|   Vivo|          21|     180|           57.0|          9|
|  Apple|          24|     214|           77.0|         19|
+-------+------------+--------+---------------+-----------+



In [27]:
df.groupBy("mobile").max().show()

+-------+------------+--------+---------------+-----------+
| mobile|max(ratings)|max(age)|max(experience)|max(family)|
+-------+------------+--------+---------------+-----------+
|     MI|           5|      42|           23.0|          5|
|   Oppo|           4|      42|           23.0|          2|
|Samsung|           5|      37|           23.0|          5|
|   Vivo|           5|      37|           23.0|          5|
|  Apple|           4|      37|           16.5|          5|
+-------+------------+--------+---------------+-----------+



In [28]:
df.groupBy("mobile").min().show()

+-------+------------+--------+---------------+-----------+
| mobile|min(ratings)|min(age)|min(experience)|min(family)|
+-------+------------+--------+---------------+-----------+
|     MI|           1|      27|            2.5|          0|
|   Oppo|           2|      22|            6.0|          0|
|Samsung|           2|      22|            2.5|          0|
|   Vivo|           3|      32|            6.0|          0|
|  Apple|           3|      22|            2.5|          0|
+-------+------------+--------+---------------+-----------+



# Aggregation

In [29]:
df.groupBy("Mobile").agg({'ratings':'sum'}).show(5,False)

+-------+------------+
|Mobile |sum(ratings)|
+-------+------------+
|MI     |28          |
|Oppo   |20          |
|Samsung|25          |
|Vivo   |21          |
|Apple  |24          |
+-------+------------+



# UDF